In [1]:
from bs4 import BeautifulSoup as bs
import requests
import json
import os
import pandas as pd
from datetime import datetime
import time 
import csv

In [5]:
class seite:
    def __init__(self,url):
        self.url = url 
        self.folderPath = "data/"
        self.isArticle = None
        self.jsonData = None


        ## get kategorie 
        self.kategorie = self.url.replace('https://www.nius.de/','').split('/')[0]

        ## create folder and files
        if os.path.isdir(self.folderPath) == False:
            os.mkdir(self.folderPath)
        
        if os.path.exists('error.log') == False:
            with open('error.log',"w")as file:
                file.write('')



    def get_article_json(self):
        
        apiPathDict = {'news':'article','politik':'article','kommentar':'article','gesellschaft':'article','wirtschaft':'article',
                       'clips':'clips','episodes':'episodes','ausland':'article','energie':'article','analyse':'article','berlin':'article',
                       'corona':'article','gastbeitrag':'article','gesellschaft':'article','interview':'article','kriminalitaet':'article','leserbrief':'article',
                       'medien':'article','nachrichten':'article','nius-live':'article','nius-originals':'article','show':'article','sport':'article',
                       'statistik':'article','unterhaltung':'article','verbrechen':'article','wirtschaft':'article',}


        if self.jsonData != None:
            return self.jsonData
        articleId = self.url.split('/')[-1]
        jsonUrl = "https://api.nius.de/"+apiPathDict[self.kategorie]+"/"+articleId
        try:
            anfrage = requests.get(jsonUrl)
        except:
            with open('error.log','a') as file:
                file.write(f'es gab ein fehler bei: {self.url},{datetime.now()}\n')
                return
        if anfrage.status_code == 200:
            self.jsonData = anfrage.json()
        else:
            print(f'no json found, statuscode: {anfrage.status_code}')
        return(self.jsonData)
    
    def save_json(self, overWrite = False):
        jsonData = self.get_article_json()
        if jsonData == None:
            return
        if overWrite == False:
            if os.path.exists(self.folderPath+jsonData['id']+'.json') == True:
                print(f"datei {self.folderPath+jsonData['id']+'.json'} existiert bereits")
                return
        with open(self.folderPath+jsonData['id']+'.json','w') as file:
            file.write(json.dumps(jsonData))
        print(f"saved {jsonData['id']}")
        return

In [ ]:
## findet alle artikel aus der sidemap und speichert sie in der Datei
import requests
import xml.etree.ElementTree as ET
import csv

# URL der Sitemap
sitemap_url = 'https://www.nius.de/sitemap.xml'

# Sitemap herunterladen
response = requests.get(sitemap_url)
response.raise_for_status()  # Überprüft, ob der Request erfolgreich war

# Sitemap parsen
sitemap = ET.fromstring(response.content)

# Alle <loc>-Tags extrahieren
loc_tags = sitemap.findall('.//{http://www.sitemaps.org/schemas/sitemap/0.9}loc')
loc_urls = [loc.text for loc in loc_tags]

csv_filename = 'sitemap_urls_29_11_24.csv'
with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    for url in loc_urls:
        csvwriter.writerow([url])

In [ ]:
## get articel links throug api 
count = 0
api = f'https://api.nius.de/articles?text=&skip0=&take=10&videoOnly=false'
text = requests.get(api).json()

while len(text['results']) != 0:
    api = f'https://api.nius.de/articles?text=&skip{count*2000}=&take=50000&videoOnly=false'
    text = requests.get(api).json()
    count +=1
    file_path = f'articel_link_throug_api/{count}.json'
    with open(file_path,'w') as file:
        file.write(json.dumps(text))

In [72]:
## extract links out of api requests
folder = 'articel_link_throug_api/'
all_links = []
for file in os.listdir(folder):
    with open(folder+file,'r') as file:
        data = json.load(file)

    for i in data['results']:
        link = f'https://www.nius.de{(i['articleRef'])}'
        all_links.append(link)
csv_filename = 'sitemap_urls_29_11_24.csv'
with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    for url in all_links:
        csvwriter.writerow([url])

In [ ]:
len(set(all_links))

In [ ]:
## rennt über die liste der urls aus der sitemap datei und speichert alle neuen
listOfUrls = []
with open ('sitemap_urls_29_11_24.csv','r') as file:
    csvFile = csv.reader(file)
    _ = [listOfUrls.append(i[0]) for i in csvFile]

df = pd.read_csv('craweld_urls.csv', index_col='id')
counter = 0

for url in listOfUrls:
    if df.url.str.contains(url,regex=False).any() == False:
            counter += 1
            seite(url).save_json()
            df.loc[len(df.index)] = [url,datetime.now()]
            if counter%50 == 0:
                df.to_csv('craweld_urls.csv')
            time.sleep(1)
df.to_csv('craweld_urls.csv')